In [6]:
import polars as pl
import numpy as np

In [7]:
rentals = pl.read_csv("dataset/synthetic_rentals.csv")
stations = pl.read_csv("dataset/synthetic_stations.csv")
users = pl.read_csv("dataset/synthetic_users.csv")


# data preprocessing 

In [11]:
display(users.sample(5))

display(rentals.sample(5))

display(stations.sample(5))


user_id,user_type,registration_date,preferred_hours,preferred_days,avg_rentals_per_month
str,str,str,str,str,i64
"""U0497""","""periodic""","""2022-07-13""","""7,8,17,18,19""","""weekday""",8
"""U0452""","""periodic""","""2023-03-03""","""10,11,12,13,14,15,16""","""weekend""",10
"""U0616""","""rare""","""2024-11-30""","""7,8,9""","""weekday""",4
"""U0174""","""periodic""","""2024-01-13""","""10,11,12,13,14,15,16""","""weekend""",10
"""U0482""","""periodic""","""2024-01-24""","""7,8,17,18,19""","""weekend""",5


rental_id,date,hour,minute,user_id,station_id,duration_minutes
str,str,i64,i64,str,str,i64
"""R034702""","""2017-01-27""",9,54,"""U0434""","""S057""",30
"""R036190""","""2017-01-28""",10,14,"""U0494""","""S082""",111
"""R036172""","""2017-01-28""",10,49,"""U0524""","""S029""",65
"""R035671""","""2017-01-28""",2,8,"""U0458""","""S022""",44
"""R016881""","""2017-01-13""",12,0,"""U0567""","""S008""",48


station_id,station_name,district,latitude,longitude,location_type,capacity,peak_hours,peak_days,usage_intensity
str,str,str,f64,f64,str,i64,str,str,f64
"""S086""","""Yeongdeungpo Station 8""","""Yeongdeungpo""",37.474077,127.068755,"""transit""",56,"""7,8,9,17,18,19,20""","""weekday""",0.88
"""S010""","""Dongdaemun Station 10""","""Dongdaemun""",37.65258,126.803168,"""mixed""",32,"""8,9,12,13,17,18,19""","""all""",0.4
"""S037""","""Seocho Station 9""","""Seocho""",37.55347,127.127222,"""recreation""",32,"""10,11,12,13,14,15,16,17""","""weekend""",0.76
"""S011""","""Eunpyeong Station 6""","""Eunpyeong""",37.453695,126.809878,"""recreation""",28,"""10,11,12,13,14,15,16,17""","""weekend""",0.66
"""S066""","""Mapo Station 2""","""Mapo""",37.464055,127.167005,"""recreation""",40,"""10,11,12,13,14,15,16,17""","""weekend""",0.64


In [16]:
rentals_stations_df = rentals.join(stations, on="station_id", how="left")  
unified_df = rentals_stations_df.join(users, on="user_id", how="left")

unified_df

rental_id,date,hour,minute,user_id,station_id,duration_minutes,station_name,district,latitude,longitude,location_type,capacity,peak_hours,peak_days,usage_intensity,user_type,registration_date,preferred_hours,preferred_days,avg_rentals_per_month
str,str,i64,i64,str,str,i64,str,str,f64,f64,str,i64,str,str,f64,str,str,str,str,i64
"""R000000""","""2017-01-01""",0,45,"""U0010""","""S034""",28,"""Jongno Station 2""","""Jongno""",37.548081,127.089129,"""business""",38,"""8,9,17,18,19""","""weekday""",0.57,"""rare""","""2022-05-12""","""10,11,12,13,14""","""weekend""",3
"""R000001""","""2017-01-01""",0,17,"""U0824""","""S031""",31,"""Seocho Station 2""","""Seocho""",37.429762,127.110368,"""residential""",19,"""7,8,18,19,20""","""all""",0.68,"""periodic""","""2024-10-08""","""7,8,17,18,19""","""weekday""",11
"""R000002""","""2017-01-01""",0,38,"""U0954""","""S035""",45,"""Seodaemun Station 5""","""Seodaemun""",37.529528,126.872021,"""transit""",38,"""7,8,9,17,18,19,20""","""weekday""",0.77,"""rare""","""2024-05-26""","""7,8,9""","""weekend""",2
"""R000003""","""2017-01-01""",0,41,"""U0047""","""S061""",55,"""Mapo Station 5""","""Mapo""",37.662058,126.992142,"""residential""",19,"""7,8,18,19,20""","""all""",0.67,"""periodic""","""2023-06-17""","""7,8,17,18,19""","""weekend""",11
"""R000004""","""2017-01-01""",0,25,"""U0908""","""S079""",21,"""Gangnam Station 10""","""Gangnam""",37.649585,126.953299,"""business""",21,"""8,9,17,18,19""","""weekday""",0.87,"""periodic""","""2022-06-10""","""10,11,12,13,14,15,16""","""weekend""",9
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""R039700""","""2017-01-31""",0,52,"""U0450""","""S028""",43,"""Eunpyeong Station 5""","""Eunpyeong""",37.549747,126.820877,"""residential""",28,"""7,8,18,19,20""","""all""",0.36,"""frequent""","""2024-07-25""","""7,8,9,12,13,17,18,19""","""weekday""",31
"""R039701""","""2017-01-31""",0,58,"""U0335""","""S098""",47,"""Yongsan Station 8""","""Yongsan""",37.655234,126.86414,"""mixed""",34,"""8,9,12,13,17,18,19""","""all""",0.42,"""frequent""","""2024-03-01""","""7,8,9,17,18,19""","""weekday""",35
"""R039702""","""2017-01-31""",0,0,"""U0628""","""S040""",52,"""Gangnam Station 2""","""Gangnam""",37.429918,127.072508,"""transit""",41,"""7,8,9,17,18,19,20""","""weekday""",0.74,"""rare""","""2024-06-17""","""7,8,9""","""weekend""",4


In [17]:
unified_df.describe()

statistic,rental_id,date,hour,minute,user_id,station_id,duration_minutes,station_name,district,latitude,longitude,location_type,capacity,peak_hours,peak_days,usage_intensity,user_type,registration_date,preferred_hours,preferred_days,avg_rentals_per_month
str,str,str,f64,f64,str,str,f64,str,str,f64,f64,str,f64,str,str,f64,str,str,str,str,f64
"""count""","""39705""","""39705""",39705.0,39705.0,"""39705""","""39705""",39705.0,"""39705""","""39705""",39705.0,39705.0,"""39705""",39705.0,"""39705""","""39705""",39705.0,"""39705""","""39705""","""39705""","""39705""",39705.0
"""null_count""","""0""","""0""",0.0,0.0,"""0""","""0""",0.0,"""0""","""0""",0.0,0.0,"""0""",0.0,"""0""","""0""",0.0,"""0""","""0""","""0""","""0""",0.0
"""mean""",null,null,11.429165,29.380909,null,null,35.656315,null,null,37.558014,126.986951,null,31.907191,null,null,0.624896,null,null,null,null,11.660773
"""std""",null,null,6.866742,17.333237,null,null,18.916603,null,null,0.089335,0.120044,null,10.204716,null,null,0.160988,null,null,null,null,10.399348
"""min""","""R000000""","""2017-01-01""",0.0,0.0,"""U0001""","""S001""",10.0,"""Dongdaemun Station 1""","""Dongdaemun""",37.420314,126.780412,"""business""",15.0,"""10,11,12,13,14,15,16,17""","""all""",0.36,"""frequent""","""2022-03-08""","""10,11,12,13,14""","""all""",1.0
"""25%""",null,null,6.0,14.0,null,null,21.0,null,null,37.480571,126.870216,null,24.0,null,null,0.5,null,null,null,null,4.0
"""50%""",null,null,11.0,29.0,null,null,31.0,null,null,37.557532,126.997293,null,32.0,null,null,0.61,null,null,null,null,8.0
"""75%""",null,null,17.0,44.0,null,null,47.0,null,null,37.651338,127.088394,null,38.0,null,null,0.75,null,null,null,null,15.0
"""max""","""R039704""","""2017-01-31""",23.0,59.0,"""U1000""","""S100""",120.0,"""Yongsan Station 9""","""Yongsan""",37.698974,127.176059,"""transit""",59.0,"""8,9,17,18,19""","""weekend""",0.99,"""rare""","""2025-03-05""","""9,10,11,12,13,14,15,16,17,18,1…","""weekend""",40.0


### Tак как если мы заменим категорилаьные числа на 0 1 2 3 4, то у нас появится некий порядок в данных, в категориальных признаках такого быть не должно, поэтому можно добавить колонки типа is_sunday, is_saturday и т.д если их не больше 20 условно (мы увеличим датасет, но там всего 40_000 строк, поэтому на работу не повлияет)

Загрузите набор данных о прокате велосипедов в Сеуле и исследуйте
его структуру.

Выполните предварительную обработку данных (например, удалите
пропущенные значения или преобразуйте категориальные переменные).

In [80]:
cat_cols = [col for col in unified_df.columns if unified_df[col].unique().shape[0] < 13]

# нет пустых значений 

In [89]:
unified_df.null_count()

rental_id,date,hour,minute,user_id,station_id,duration_minutes,station_name,district,latitude,longitude,location_type,capacity,peak_hours,peak_days,usage_intensity,user_type,registration_date,preferred_hours,preferred_days,avg_rentals_per_month
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [86]:
def get_df_with_cat_binary_cols(cat_cols: list, df: pl.DataFrame) -> pl.DataFrame:
      df = df.with_columns(
          [pl.when(pl.col(cat_col) == val)
          .then(1)
          .otherwise(0)
          .alias(f"{cat_col}_{val}") for cat_col in cat_cols for val in
        df[cat_col].unique().to_list()]
        )
      return df

unifield_df_with_binary_cat_cols = get_df_with_cat_binary_cols(cat_cols, df=unified_df)

In [87]:
unifield_df_with_binary_cat_cols

rental_id,date,hour,minute,user_id,station_id,duration_minutes,station_name,district,latitude,longitude,location_type,capacity,peak_hours,peak_days,usage_intensity,user_type,registration_date,preferred_hours,preferred_days,avg_rentals_per_month,district_Yongsan,district_Gangnam,district_Seodaemun,district_Yeongdeungpo,district_Seocho,district_Mapo,district_Dongdaemun,district_Jongno,district_Nowon,district_Eunpyeong,location_type_residential,location_type_mixed,location_type_recreation,location_type_transit,location_type_business,"peak_hours_8,9,12,13,17,18,19","peak_hours_7,8,9,17,18,19,20","peak_hours_10,11,12,13,14,15,16,17","peak_hours_7,8,18,19,20","peak_hours_8,9,17,18,19",peak_days_weekday,peak_days_weekend,peak_days_all,user_type_rare,user_type_frequent,user_type_periodic,"preferred_hours_7,8,17,18,19","preferred_hours_16,17,18,19,20","preferred_hours_9,10,11,12,13,14,15,16,17,18,19,20","preferred_hours_10,11,12,13,14,15,16","preferred_hours_7,8,9,17,18,19","preferred_hours_7,8,9,12,13,17,18,19","preferred_hours_10,11,12,13,14","preferred_hours_7,8,9",preferred_days_weekday,preferred_days_weekend,preferred_days_all
str,str,i64,i64,str,str,i64,str,str,f64,f64,str,i64,str,str,f64,str,str,str,str,i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""R000000""","""2017-01-01""",0,45,"""U0010""","""S034""",28,"""Jongno Station 2""","""Jongno""",37.548081,127.089129,"""business""",38,"""8,9,17,18,19""","""weekday""",0.57,"""rare""","""2022-05-12""","""10,11,12,13,14""","""weekend""",3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
"""R000001""","""2017-01-01""",0,17,"""U0824""","""S031""",31,"""Seocho Station 2""","""Seocho""",37.429762,127.110368,"""residential""",19,"""7,8,18,19,20""","""all""",0.68,"""periodic""","""2024-10-08""","""7,8,17,18,19""","""weekday""",11,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0
"""R000002""","""2017-01-01""",0,38,"""U0954""","""S035""",45,"""Seodaemun Station 5""","""Seodaemun""",37.529528,126.872021,"""transit""",38,"""7,8,9,17,18,19,20""","""weekday""",0.77,"""rare""","""2024-05-26""","""7,8,9""","""weekend""",2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0
"""R000003""","""2017-01-01""",0,41,"""U0047""","""S061""",55,"""Mapo Station 5""","""Mapo""",37.662058,126.992142,"""residential""",19,"""7,8,18,19,20""","""all""",0.67,"""periodic""","""2023-06-17""","""7,8,17,18,19""","""weekend""",11,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0
"""R000004""","""2017-01-01""",0,25,"""U0908""","""S079""",21,"""Gangnam Station 10""","""Gangnam""",37.649585,126.953299,"""business""",21,"""8,9,17,18,19""","""weekday""",0.87,"""periodic""","""2022-06-10""","""10,11,12,13,14,15,16""","""weekend""",9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""R039700""","""2017-01-31""",0,52,"""U0450""","""S028""",43,"""Eunpyeong Station 5""","""Eunpyeong""",37.549747,126.820877,"""residential""",28,"""7,8,18,19,20""","""all""",0.36,"""frequent""","""2024-07-25""","""7,8,9,12,13,17,18,19""","""weekday""",31,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0
"""R039701""","""2017-01-31""",0,58,"""U0335""","""S098""",47,"""Yongsan Station 8""","""Yongsan""",37.655234,126.86414,"""mixed""",34,"""8,9,12,13,17,18,19""","""all""",0.42,"""frequent""","""2024-03-01""","""7,8,9,17,18,19""","""weekday""",35,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0
"""R039702""","""2017-01-31""",0,0,"""U0628""","""S040""",52,"""Gangnam Station 2""","""Gangnam""",37.429918,127.072508,"""transit""",41,"""7,8,9,17,18,19,20""","""weekday""",0.74,"""rare""","""2024-06-17""","""7,8,9""","""weekend""",4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,

In [105]:
unified_df_datetime = unifield_df_with_binary_cat_cols.with_columns(
      pl.datetime(
          year=pl.col("date").str.strptime(pl.Date,
  "%Y-%m-%d").dt.year(),
          month=pl.col("date").str.strptime(pl.Date,
  "%Y-%m-%d").dt.month(),
          day=pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").dt.day(),
          hour=pl.col("hour"),
          minute=pl.col("minute")
      ).alias("datetime")
  )

unified_df_timestamp = unified_df_datetime.with_columns(
    pl.col("datetime").dt.epoch(time_unit="s").alias("datetime").alias("unix_timestamp")
)

# work with numpy

In [106]:
def get_numpy_col_from_num(df: pl.DataFrame, col: str) -> np.ndarray:
    return df.select(col).to_numpy().flatten()

duration_vec = get_numpy_col_from_num(unified_df_timestamp, "duration_minutes")
timestamp_vec = get_numpy_col_from_num(unified_df_timestamp, "unix_timestamp")
capacity_vec = get_numpy_col_from_num(unified_df_timestamp, "capacity")

timestamp_vec

array([1483231500, 1483229820, 1483231080, ..., 1485820800, 1485821580,
       1485822660], shape=(39705,))

In [ ]:
# 2. Create matrices representing different aspects of the data
       # Time-based matrix (hour x day patterns)
       unique_hours = np.unique(hour_vector)
       days_sample = 30  # First 30 days
       time_matrix = np.zeros((len(unique_hours), days_sample))

       # Fill matrix with rental counts per hour per day
       for day in range(days_sample):
           day_data = unified_df.filter(
               (pl.col("date") >= "2017-01-01") &
               (pl.col("date") <= "2017-01-30")
           ).group_by("hour").agg(pl.count().alias("count"))

           if len(day_data) > 0:
               hours_data = day_data.select("hour").to_numpy().flatten()
               counts_data =
       day_data.select("count").to_numpy().flatten()
               for i, hour in enumerate(hours_data):
                   if hour < len(unique_hours):
                       time_matrix[hour, day] = counts_data[i]

       print(f"Time matrix shape: {time_matrix.shape}")
       print(f"Sample of time matrix:\n{time_matrix[:5, :5]}")

       # Station metrics matrix (stations x metrics)
       station_ids =
       unified_df.select("station_id").unique().to_numpy().flatten()
       n_stations = min(len(station_ids), 20)  # Limit to first 20 
       stations for demo

       station_metrics = np.zeros((n_stations, 4))  # 4 metrics: 
       capacity, usage_intensity, avg_duration, total_rentals

       for i in range(n_stations):
           station_data = unified_df.filter(pl.col("station_id") ==
       station_ids[i])
           if len(station_data) > 0:
               station_metrics[i, 0] =
       station_data.select("capacity").mean().item()
               station_metrics[i, 1] =
       station_data.select("usage_intensity").mean().item()
               station_metrics[i, 2] =
       station_data.select("duration_minutes").mean().item()
               station_metrics[i, 3] = len(station_data)

       print(f"\nStation metrics matrix shape: {station_metrics.shape}")
       print(f"Columns: [capacity, usage_intensity, avg_duration, 
       total_rentals]")
       print(f"Sample station metrics:\n{station_metrics[:5]}")

In [ ]:
  print("=== INDEXING AND SLICING OPERATIONS ===")

       # Extract morning hours (7-11) from time matrix
       morning_hours = time_matrix[7:12, :]
       print(f"Morning hours data shape: {morning_hours.shape}")
       print(f"Morning rentals (first 5 days):\n{morning_hours[:, :5]}")

       # Extract weekend data (assuming days 5,6,12,13,19,20,26,27 are 
       weekends in first month)
       weekend_days = [5, 6, 12, 13, 19, 20, 26, 27]
       weekend_data = time_matrix[:, weekend_days]
       print(f"\nWeekend data shape: {weekend_data.shape}")

       # Extract evening peak hours (17-20) for specific days
       evening_peak = time_matrix[17:21, 0:7]  # First week evening data
       print(f"Evening peak (first week) shape: {evening_peak.shape}")

       # Slice station metrics for high-usage stations
       high_usage_mask = station_metrics[:, 1] > 0.7  # usage_intensity > 
       0.7
       high_usage_stations = station_metrics[high_usage_mask]
       print(f"\nHigh usage stations shape: {high_usage_stations.shape}")
       print(f"High usage stations metrics:\n{high_usage_stations}")

       # Extract specific metrics columns
       capacity_column = station_metrics[:, 0]  # All capacities
       duration_column = station_metrics[:, 2]  # All average durations
       print(f"\nCapacity values for first 10 stations: 
       {capacity_column[:10]}")
       print(f"Duration values for first 10 stations: 
       {duration_column[:10]}")

In [ ]:
    # 4. Vector operations (addition, subtraction, scalar 
       multiplication)
       print("=== VECTOR OPERATIONS ===")

       # Take smaller samples for demonstration
       duration_sample = duration_vector[:1000]
       capacity_sample = capacity_vector[:1000]
       usage_sample = usage_intensity_vector[:1000]

       # Vector addition - combine duration and capacity effects
       combined_metric = duration_sample + capacity_sample
       print(f"Combined metric (duration + capacity) sample: 
       {combined_metric[:10]}")

       # Vector subtraction - difference between capacity and usage
       capacity_gap = capacity_sample - (usage_sample * 100)  # Scale 
       usage for comparison
       print(f"Capacity gap sample: {capacity_gap[:10]}")

       # Scalar multiplication - normalize durations to hours
       duration_hours = duration_sample / 60.0
       print(f"Duration in hours sample: {duration_hours[:10]}")

       # Element-wise multiplication
       efficiency_metric = duration_sample * usage_sample
       print(f"Efficiency metric (duration × usage) sample: 
       {efficiency_metric[:10]}")

       # Vector norms and statistics
       print(f"\nVector statistics:")
       print(f"Duration vector L2 norm: 
       {np.linalg.norm(duration_sample):.2f}")
       print(f"Duration vector mean: {np.mean(duration_sample):.2f}")
       print(f"Duration vector std: {np.std(duration_sample):.2f}")

       # Dot product between vectors
       duration_capacity_dot = np.dot(duration_sample, capacity_sample)
       print(f"Dot product (duration • capacity): 
       {duration_capacity_dot:.2f}")
       

In [ ]:
 # 5. Broadcasting operations with different shaped arrays
       print("=== BROADCASTING OPERATIONS ===")

       # Create arrays of different shapes for broadcasting
       hourly_pattern = np.array([0.5, 0.7, 0.8, 0.9, 1.2, 1.5, 1.8, 1.0,
       0.8, 0.6])  # 10 hours pattern
       daily_multiplier = np.array([[1.0], [1.2], [0.8], [1.1], [0.9]])  #
        5 days multiplier (column vector)

       print(f"Hourly pattern shape: {hourly_pattern.shape}")
       print(f"Daily multiplier shape: {daily_multiplier.shape}")

       # Broadcasting: multiply hourly pattern by daily multiplier
       demand_matrix = daily_multiplier * hourly_pattern
       print(f"\nDemand matrix shape after broadcasting: 
       {demand_matrix.shape}")
       print(f"Demand matrix:\n{demand_matrix}")

       # Broadcasting with station metrics
       station_base_capacity = station_metrics[:5, 0]  # First 5 stations 
       capacity
       time_adjustment = np.array([0.8, 1.0, 1.2, 0.9])  # 4 time periods

       print(f"\nStation capacity shape: {station_base_capacity.shape}")
       print(f"Time adjustment shape: {time_adjustment.shape}")

       # Broadcast to create time-adjusted capacity matrix
       adjusted_capacity = station_base_capacity.reshape(-1, 1) *
       time_adjustment
       print(f"Adjusted capacity shape: {adjusted_capacity.shape}")
       print(f"Adjusted capacity matrix:\n{adjusted_capacity}")

       # More complex broadcasting: add weather effect
       weather_effect = np.array([1.1, 0.9, 1.0, 1.2, 0.8])  # 5 weather 
       conditions
       station_weather_capacity = station_base_capacity.reshape(-1, 1, 1)
       * time_adjustment.reshape(1, -1, 1) * weather_effect.reshape(1, 1,
       -1)

       print(f"\nStation-Time-Weather capacity shape: 
       {station_weather_capacity.shape}")
       print(f"Sample of 3D broadcasted array (station 
       0):\n{station_weather_capacity[0]}")

       # Broadcasting with boolean operations
       high_capacity_mask = station_base_capacity > 30
       print(f"\nHigh capacity stations mask: {high_capacity_mask}")

       # Apply mask with broadcasting
       filtered_adjusted = np.where(high_capacity_mask.reshape(-1, 1),
       adjusted_capacity, 0)
       print(f"Filtered adjusted capacity (high capacity stations 
       only):\n{filtered_adjusted}")